In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 데이터 그냥 확인절차

In [3]:
df = pd.read_excel("/Users/leesh970930/Desktop/data_team/all_data.xlsx", delimiter=",")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18554 entries, 0 to 18553
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  18554 non-null  int64 
 1   contents    18554 non-null  object
 2   time        18554 non-null  object
 3   keyword     18554 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.9+ KB


In [4]:
df.head()

,Unnamed,keyword,title,nickname,datetime
0,0,음악,적재(정재원) - The Door [음악/노래/추천/공유],양양,2021. 12. 1. 14:17
1,1,음악,"음악작업용 외장하드 2테라, 내장SSD랑 속도 차이 거의 없을 무",MosicA,2021. 9. 21. 19:00
2,2,음악,미디작곡학원 나만의 음악을 만들고 싶다면,조감자\n(stella_lucy),2021. 11. 30. 14:04
3,3,음악,실용음악 선택이 미래를 좌우!,용뇽,2021. 12. 2. 11:13
4,4,음악,[음악교육신문] 피아니스트 이정아,음악교육신문,2021. 11. 12. 13:57


In [6]:
list(df["title"][:5])

['적재(정재원) - The Door [음악/노래/추천/공유]',
 '음악작업용 외장하드 2테라, 내장SSD랑 속도 차이 거의 없을 무',
 '미디작곡학원 나만의 음악을 만들고 싶다면',
 '실용음악 선택이 미래를 좌우!',
 '[음악교육신문] 피아니스트 이정아']

In [6]:
#df2 = df[::3]  #3번째 지나면 1개씩 뽑기 (0, 3, 6...)
#df2.info()

In [8]:
#df2.head()   

# 데이터 전처리 (url,숫자,영어,#,특수문자제거, 정규화 )

In [9]:
df = pd.read_excel("/Users/leesh970930/Desktop/data_team/all_data.xlsx", delimiter=",")

In [10]:
data = df.drop(columns=['Unnamed'], axis=1)

In [11]:
data.head()

,keyword,title,nickname,datetime
0,음악,적재(정재원) - The Door [음악/노래/추천/공유],양양,2021. 12. 1. 14:17
1,음악,"음악작업용 외장하드 2테라, 내장SSD랑 속도 차이 거의 없을 무",MosicA,2021. 9. 21. 19:00
2,음악,미디작곡학원 나만의 음악을 만들고 싶다면,조감자\n(stella_lucy),2021. 11. 30. 14:04
3,음악,실용음악 선택이 미래를 좌우!,용뇽,2021. 12. 2. 11:13
4,음악,[음악교육신문] 피아니스트 이정아,음악교육신문,2021. 11. 12. 13:57


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   1674 non-null   object
 1   title     1674 non-null   object
 2   nickname  1674 non-null   object
 3   datetime  1674 non-null   object
dtypes: object(4)
memory usage: 52.4+ KB


In [13]:
#conents에 해당하는 부분만 'docs'라는 리스트로 만듦

docs = list(data['title'])

In [14]:
docs

['적재(정재원) - The Door [음악/노래/추천/공유]',
 '음악작업용 외장하드 2테라, 내장SSD랑 속도 차이 거의 없을 무',
 '미디작곡학원 나만의 음악을 만들고 싶다면',
 '실용음악 선택이 미래를 좌우!',
 '[음악교육신문] 피아니스트 이정아',
 '캐릭터라이선싱페어2021에서 만난 음악교육 애니메이션 고둥둥심포니',
 '[책] 상페의 음악 / 당신은 무엇을 동경하십니까?',
 '[유아음악] 세계적인 유아음악 프로그램 "킨더뮤직(Kindermusik)" ',
 '[BDCABU]재즈 역사상 가장 뛰어난 보컬 빌리 홀리데이의 음악!',
 '실용음악학과 서울권 진학을 알아본다면',
 'LP 레코드로 한국대중음악을 즐기기',
 '소란(SORAN) - 내꺼라면 [음악/노래/추천/공유]',
 '[음악장비]묵직한 세츄레이션의 Acme Audio Opticom XLA-500  Series Tube Compressor',
 '실용음악학과 현실적인 활동과 취업방향 잡고싶다면!',
 'EMI 시네마 클래식 - 영화로 즐기는 클래식 음악',
 '[음악교육신문] 「피아노 석세스 올인원」 출간 (예솔출판사)',
 '[2022 살롱드파주X그레이스제이피아노] 음악캠프 (플룻/피아노/성악)',
 '보컬레슨 추천 - 음악 전문 온라인 교육 플레이샵 ',
 '실용음악 비전공자로 성공하는 방법',
 '[킨더뮤직] 영유아에게 백색소음보다 음악이 더 좋은 이유?',
 '서울예대 실용음악과 2022 수시 1차 4명 합격',
 '롤링 스톤 매거진(선정) 역대 위대한 음악 500(개정판) TOP 10 ',
 '10CM(십센치) - 어제 너는 나를 버렸어 [음악/노래/추천/공유]',
 '[음악장비]깔끔한 니브 색채의 원조 AMS NEVE 1073 LB Mic Preamp 500 시리즈 마이크 프리앰프',
 '음악에서 무엇을 들어 낼 것인가 - 애런 코플랜드',
 '[음악교육신문] 한세대학교 일반대학원 음악과장 유영재 교수',
 '바흐(J.S.Bach) - The #1 Bach Album

In [15]:
import re
import requests
from bs4 import BeautifulSoup


from konlpy.tag import Okt 
okt = Okt()

In [16]:
# http로 시작되는 url제거
remove_url = [re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", str(content)) for content in docs]

# 숫자 제거
remove_num = [re.sub(r'\d+',' ', str(content)) for content in remove_url]

# 영어 제거 
remove_alpha = [re.sub('[a-zA-Z]' , ' ', str(content)) for content in remove_num]

# Hashtag 제거
remove_tag = [re.sub('[#]+[0-9a-zA-Z_]+', ' ', str(content)) for content in remove_alpha]

# 특수문자 제거
remove_special = [re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', ' ', str(content)) for content in remove_tag]

#정규식(Regular Expression)이용해서 정규화
lawdata = [re.sub(r"[^가-힣A-Za-z0-9]", " ", str(content)) for content in remove_special]


   # 쓰레기 단어 제거
   #text = re.sub('[&]+[a-z]+', ' ', text)

   # 특정 문자 제거
   #for i in ['!','@','#','*'] :
       #new_id = new_id.replace(i, "")

   # 띄어쓰기 제거
   #text = text.replace('\n',' ')

In [17]:
lawdata[:20]

['적재 정재원              음악 노래 추천 공유 ',
 '음악작업용 외장하드  테라  내장   랑 속도 차이 거의 없을 무',
 '미디작곡학원 나만의 음악을 만들고 싶다면',
 '실용음악 선택이 미래를 좌우 ',
 ' 음악교육신문  피아니스트 이정아',
 '캐릭터라이선싱페어 에서 만난 음악교육 애니메이션 고둥둥심포니',
 ' 책  상페의 음악   당신은 무엇을 동경하십니까 ',
 ' 유아음악  세계적인 유아음악 프로그램  킨더뮤직               ',
 '        재즈 역사상 가장 뛰어난 보컬 빌리 홀리데이의 음악 ',
 '실용음악학과 서울권 진학을 알아본다면',
 '   레코드로 한국대중음악을 즐기기',
 '소란          내꺼라면  음악 노래 추천 공유 ',
 ' 음악장비 묵직한 세츄레이션의                                                 ',
 '실용음악학과 현실적인 활동과 취업방향 잡고싶다면 ',
 '    시네마 클래식   영화로 즐기는 클래식 음악',
 ' 음악교육신문   피아노 석세스 올인원  출간  예솔출판사 ',
 '   살롱드파주 그레이스제이피아노  음악캠프  플룻 피아노 성악 ',
 '보컬레슨 추천   음악 전문 온라인 교육 플레이샵 ',
 '실용음악 비전공자로 성공하는 방법',
 ' 킨더뮤직  영유아에게 백색소음보다 음악이 더 좋은 이유 ']

# 형태소 분석 관련 코드(여기서는 okt.nouns 이용해서, 토크나이즈 후 명사만 골라냄)

In [18]:
#morphs는 형태소 단위로 구문을 분석(형태소 단위로 토크나이즈)

lawdata_tokened_morphs = []

for doc in lawdata:
    newdoc = okt.morphs(doc)
    lawdata_tokened_morphs.append(newdoc)

In [19]:
lawdata_tokened_morphs[:2]

[['적재', '정재원', '음악', '노래', '추천', '공유'],
 ['음악', '작업', '용', '외장하드', '테라', '내장', '랑', '속도', '차이', '거의', '없을', '무']]

In [20]:
#명사만 추출해줌(토크나이즈해서 리스트 형태로 명사만 추출)

lawdata_tokened_nouns = []

for doc in lawdata:
    newdoc = okt.nouns(doc)
    lawdata_tokened_nouns.append(newdoc)

In [20]:
lawdata_tokened_nouns[:2]

[['개', '뿐', '한지', '얼마', '은', '기초', '단계'],
 ['아이패드', '타블렛', '뭐', '클래스', '그거', '형', '누나']]

In [21]:
#각 품사를 태깅하는 역할. 
#품사를 태깅한다는 것은 주어진 텍스트를 형태소 단위로 나누고, 나눠진 각 형태소를 그에 해당하는 품사와 함께 리스트화 하는 것을 의미한다. 
#형태소 분석으로, 형태소 단위 토크나이징
#pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True)]


pos = lambda d: ['/'.join(p) for p in okt.pos(d, stem=True, norm=True) 
                 if ('Noun' in p or 'Adjective' in p or 'Verb' in p or 'Adverb' in p) 
                 and '보다' not in p and '하다' not in p]
lawdata_tokened = [pos(doc) for doc in lawdata]

In [22]:
lawdata_tokened[:2]

[['적재/Noun', '정재원/Noun', '음악/Noun', '노래/Noun', '추천/Noun', '공유/Noun'],
 ['음악/Noun',
  '작업/Noun',
  '용/Noun',
  '외장하드/Noun',
  '테라/Noun',
  '내장/Noun',
  '속도/Noun',
  '차이/Noun',
  '거의/Noun',
  '없다/Adjective',
  '무/Noun']]

In [23]:
print(lawdata_tokened[1])
lawdata_tokened[1][0].split('/')

['음악/Noun', '작업/Noun', '용/Noun', '외장하드/Noun', '테라/Noun', '내장/Noun', '속도/Noun', '차이/Noun', '거의/Noun', '없다/Adjective', '무/Noun']


['음악', 'Noun']

In [24]:
lawdata_tokened_filtered = []

for doc in lawdata_tokened:
    newdoc = [d for d in doc if d.split('/')[1] == 'Noun' or d.split('/')[1] == 'Adjective' or d.split('/')[1] == 'Adverb']
    lawdata_tokened_filtered.append(newdoc)

In [25]:
lawdata_tokened_filtered[:2]

[['적재/Noun', '정재원/Noun', '음악/Noun', '노래/Noun', '추천/Noun', '공유/Noun'],
 ['음악/Noun',
  '작업/Noun',
  '용/Noun',
  '외장하드/Noun',
  '테라/Noun',
  '내장/Noun',
  '속도/Noun',
  '차이/Noun',
  '거의/Noun',
  '없다/Adjective',
  '무/Noun']]

In [26]:
#명사만 추출하는 함수 정의(토크나이즈해서 명사만 추출)

import string
def noun_process(text):
    nouns = okt.nouns(text)
    #nouns = [word for word in nouns if len(word) > 1]
    return nouns

In [27]:
print(lawdata[0])
print(noun_process(lawdata[0]))

적재 정재원              음악 노래 추천 공유 
['적재', '정재원', '음악', '노래', '추천', '공유']


# CountVectorizer해서 word of bags 만듦 -> 문서전체를 doc-term matrix로 변환

In [28]:
#CountVectorizer: 토큰이 문서별로 몇 번 등장했는지 행렬로 정리해주는 모듈
#CountVectorizer해서 word of bags만들어주는 'bow_transformer' 함수 정의

bow_transformer = CountVectorizer(analyzer=noun_process).fit(lawdata)

In [29]:
#총 단어갯수

len(bow_transformer.vocabulary_)

3492

In [30]:
# 0번째줄 문장의 단어는 어느 위치의 벡터인가?, 몇 번 나오는가?

bow_0 = bow_transformer.transform([lawdata[0]])
print(bow_0)

  (0, 204)	1
  (0, 467)	1
  (0, 2227)	1
  (0, 2432)	1
  (0, 2499)	1
  (0, 2766)	1


In [31]:
# 이제 모든 lawdata를 변형

cv_lawdata= bow_transformer.transform(lawdata)

In [32]:
# CountVectorizer 적용한 lawdata 확인

print('Shape of Sparse Matrix: ', cv_lawdata.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata.nnz)

Shape of Sparse Matrix:  (1674, 3492)
Amount of Non-Zero occurrences:  12765


In [33]:
# 그런데 여기서 주의해야할 것 하나.
# docs는 현재 document-term matrix 로 구성됨
# NMF를 적용하기 위해서는 term-document matrix가 필요하다.
# 즉, 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.

type(cv_lawdata)

scipy.sparse.csr.csr_matrix

In [34]:
# compressed sparse(희소행렬: 행렬의 값이 대부분 0일때) row matrix인 docs를 transpose 한다. (행열바꾸기)

cv_lawdata_trans = cv_lawdata.transpose()
type(cv_lawdata_trans) 

print('Shape of Sparse Matrix: ', cv_lawdata_trans.shape)
print('Amount of Non-Zero occurrences: ', cv_lawdata_trans.nnz)

Shape of Sparse Matrix:  (3492, 1674)
Amount of Non-Zero occurrences:  12765


# NMF 모델 객체 생성

In [35]:
# NMF 모델 객체 생성
# n_topics: 매칭해줄 분류객체(베이킹, 요리..)

from sklearn.decomposition import NMF

n_topics = 25
model = NMF(n_components=n_topics)

In [36]:
# NMF 모델 학습
W = model.fit_transform(cv_lawdata_trans)

/Users/leesh970930/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [37]:
H = model.components_

In [38]:
H.shape

(25, 1674)

In [39]:
# index 0번 문서의 클러스터링 결과 점수

print(H[:,0])

[0.         0.         0.         0.         0.         0.21823515
 0.         0.         0.         0.         0.         0.
 0.         0.58325831 0.         0.         0.         0.
 0.         0.         0.         0.         0.05135359 0.
 0.        ]


In [40]:
# 전체 글 별로 best점수에 해당하는 클러스터 label결정

pred_labels = H.argmax(axis=0)

In [41]:
# label예측한 글 개수 = 원래 글 개수

len(pred_labels)

1674

In [42]:
#10번째글까지 예측한 각 글에 대한 label 번호

pred_labels[:10]

array([13,  5,  5, 11,  5,  5,  5,  5,  5, 20])

In [43]:
#1번째글은 15번째(0.00913391), 2번째글은 23번째(0.15965101), 3번째글은 16번째(0.13495328) 값인 것 확인!

H[:,:3]

array([[0.        , 0.00439797, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.00907763, 0.00114899],
       [0.        , 0.        , 0.00604478],
       [0.21823515, 0.2308517 , 0.23272173],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.06472119],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.15691424],
       [0.        , 0.        , 0.        ],
       [0.58325831, 0.02501105, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.0008023 , 0.01344818],
       [0.        , 0.00184227, 0.        ],
       [0.        , 0.        , 0.        ],
       [0.

In [44]:
# 원래데이터('data')에서 'keywords'열만 뽑아서 'target'에 저장
target = list(data["keyword"])
print(target)

['음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악', '음악

In [49]:
# 'target'값(문자열)을 순서대로 '0 ~ 24'와 같은 숫자형으로 변환

num_target = []

for i in target:
    temp = i.replace('음악','0').replace('베이킹','1').replace('외국어','2').replace('요리','3').replace('운동','4').replace('라이프','5').replace('공예','6').replace('드로잉','7').replace('디지털드로잉','8')
    num_target.append(int(temp))
    
    
print(num_target)

ValueError: invalid literal for int() with base 10: '디지털7'

In [50]:
len(num_target)

1453

In [51]:
from sklearn.metrics import confusion_matrix

#가로: 예측 클러스터 / 세로: 실제 클래스
pd.DataFrame(confusion_matrix(num_target, pred_labels))

ValueError: Found input variables with inconsistent numbers of samples: [1453, 1674]

In [48]:
cmatrix = confusion_matrix(num_target, pred_labels)

#10번째열 까지만 출력
cmatrix[:,:10]

array([[  0,   1,   0,   0,   1,   0,   0,   3,   0,   0],
       [ 17,   7,   2,   2,  66,   3,  10,  14,   2,   3],
       [ 49,   8,   7,   4,  16,  13,  12,  19,   4,   7],
       [155,   9,   5,   8,  28,  39,  25,  19,   5,   5],
       [136,   9,  11,  16,  77,  52,  15,  22,  16,  10],
       [ 67,   6,   9,   4,  17,  29,  12,  24,   5,   6],
       [102,   8,   9,  18,  29,  19,  24,  18,   6,   6],
       [197,   6,  14,   9,  55,  53,  38,  18,  12,   8],
       [ 84,   4,   7,   9,  35, 612,  18,  12,   1,   5],
       [225,  14,  48,  34,  67,  51,  40,  26,  12,  43],
       [205,  10,  10,   4,  28,  53,  16,  18,   0,   2],
       [205,  27,  25,  21, 119,  43,  48,  31,  19,  33],
       [152,  11,  41,  22, 112,  19,  18,  24,   9,  74],
       [114,   6,  50,  20,  51,  15,  24,  20,  40,  52],
       [107,   7,  31,  21,  69,   7,  24,  26,  38,  21],
       [ 36,   0,  33,   1,  18,   0,   2,  14,   9,  27],
       [ 67,  89,  25,   5,  40,   6,  16,  14,   9,  20

In [49]:
# 25개
labels_25 = list(cmatrix.argmax(axis=0))
print(labels_25)

[9, 16, 20, 20, 21, 8, 11, 21, 18, 21, 16, 21, 10, 17, 21, 22, 4, 14, 19, 7, 13, 6, 7, 4, 9]


In [50]:
#labels_25로 매칭했을 때, 전체 클러스터링 정확도 계산 과정(성능확인 위해서)

i = 0
sum_result = 0
for c in cmatrix.transpose():
    print(c)
    print('label:', labels_25[i])
    print('value:', c[labels_25[i]])
    i += 1
    sum_result += c[labels_25[i]]
    

[  0  17  49 155 136  67 102 197  84 225 205 205 152 114 107  36  67 155
  77 101  73 144 109  24  11]
label: 9
value: 225
[ 1  7  8  9  9  6  8  6  4 14 10 27 11  6  7  0 89 15  8  5 15 11  4  0
  1]
label: 16
value: 89
[ 0  2  7  5 11  9  9 14  7 48 10 25 41 50 31 33 25 14 31 49 70 42 17 11
 10]
label: 20
value: 70
[  0   2   4   8  16   4  18   9   9  34   4  21  22  20  21   1   5   5
   4  59 453  39 103  25   1]
label: 20
value: 453
[  1  66  16  28  77  17  29  55  35  67  28 119 112  51  69  18  40  32
  24  26  23 121  62   6   2]
label: 21
value: 121
[  0   3  13  39  52  29  19  53 612  51  53  43  19  15   7   0   6  11
  32  16   4   8  20   2   0]
label: 8
value: 612
[ 0 10 12 25 15 12 24 38 18 40 16 48 18 24 24  2 16 33 31  8 13 48 21 10
  0]
label: 11
value: 48
[ 3 14 19 19 22 24 18 18 12 26 18 31 24 20 26 14 14 35 30 19 21 55 41  5
  2]
label: 21
value: 55
[  0   2   4   5  16   5   6  12   1  12   0  19   9  40  38   9   9 307
 465  25  55  16  20   3   1]
label: 18
v

IndexError: list index out of range

In [51]:
# 맞은데이터 개수

print(sum_result)

748


In [52]:
# 맞은 비율

sum_result/len(num_target)

0.0403147569257303